In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from PIL import Image
from datetime import datetime
import re

# Load the model and processor
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct"
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [14]:
!pip install --upgrade torch transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 88.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [12]:
image = Image.open("/content/birthcerfiticate-ezgif.com-avif-to-jpg-converter.jpg")
image = image.resize((image.width // 2, image.height // 2))
# Define the user message
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {
                "type": "text",
                "text": "Extract Name, Date of Birth, Gender, Place of Birth, Father's Name, Mother's Name, Registration Number, Date of Registration, and Address."
            }
        ]
    }
]

# Create a text prompt with the updated task
text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

# Process both the text and image inputs
inputs = processor(
    text=[text_prompt],
    images=[image],
    padding=True,
    return_tensors="pt"
)

# Move inputs to CUDA
inputs = inputs.to("cuda")

# Generate output text
output_ids = model.generate(**inputs, max_new_tokens=512)

# Decode the generated output text
output_text = processor.batch_decode(
    output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

# Print the entire output text for debugging
print(f"Generated Text: {output_text[0]}")

# Define regex patterns for extracting personal details
patterns = {
    "Name": r"Name:\s*([A-Za-z\s]+)",
    "Date of Birth": r"Date of Birth:\s*([\d]{2}-[A-Za-z]{3}-[\d]{4})",
    "Gender": r"Gender:\s*(Male|Female|Other)",
    "Place of Birth": r"Place of Birth:\s*([A-Za-z\s]+)",
    "Father's Name": r"Father's Name:\s*([A-Za-z\s]+)",
    "Mother's Name": r"Mother's Name:\s*([A-Za-z\s]+)",
    "Registration Number": r"Registration Number:\s*([\d]+)",
    "Date of Registration": r"Date of Registration:\s*([\d]{2}-[A-Za-z]{3}-[\d]{4})",
    "Address": r"Address:\s*([A-Za-z0-9\s,]+|Not Available)"
}

# Extract details using regex
extracted_details = {}
for key, pattern in patterns.items():
    match = re.search(pattern, output_text[0])
    extracted_details[key] = match.group(1) if match else "Not found"

# Display the extracted details
# for key, value in extracted_details.items():
#     print(f"{key}: {value}")

Generated Text: system
You are a helpful assistant.
user
Extract Name, Date of Birth, Gender, Place of Birth, Father's Name, Mother's Name, Registration Number, Date of Registration, and Address.
assistant
The birth certificate contains the following information:

- Name: ERAM FATHIMA
- Date of Birth: 07/03/2015
- Gender: Female
- Place of Birth: K.C. General Hospital
- Name of Father: WASEEM ARIF
- Name of Mother: NAZIYA PARVEEN
- Address: NO.25 5TH CROSS SHAMANNA GARDEN OLD GUDDADAHALLI BENGALURU-26
- Registration Number: KCG/B/2015/00050/000845
- Date of Registration: 23/03/2015
- Date of Issue: 23/03/2015
- Signature of Issuing Authority: (Signature of the issuing authority)

The certificate is issued under Section 12/17 of the RBD Act, 1969, and Rule 8/13 of the KRBD Rules, 1999.


In [6]:
import torch
torch.cuda.empty_cache()

In [5]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True



In [8]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 2            |        cudaMalloc retries: 3         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   5037 MiB |  13508 MiB |  27716 MiB |  22679 MiB |\n|       from large pool |   4993 MiB |  13465 MiB |  27627 MiB |  22633 MiB |\n|       from small pool |     43 MiB |     86 MiB |     89 MiB |     45 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   5037 MiB |  13508 MiB |  27716 MiB |  22679 MiB |\n|       from large pool |   4993 MiB |  13465 MiB |